In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <cstddef>
#include <torch/torch.h>
#include <tuple>
#include <c10/util/Optional.h>

## step 1: create mock data x and y

In [3]:
std::pair<torch::Tensor, torch::Tensor> synthetic_data(torch::Tensor true_w, float true_b, int64_t num_samples) {

    auto X = torch::normal(0.0, 1.0, {num_samples, true_w.size(0)});
    auto y = torch::matmul(X, true_w) + true_b;
    y += torch::normal(0.0, 0.01, y.sizes());
    y = torch::reshape(y, {-1, 1});

    //return torch::cat({X, y}, 1);
    return {X, y};
 }

In [4]:
auto options = torch::TensorOptions().dtype(torch::kFloat).device(torch::kCPU);

// Generating the Dataset
torch::Tensor true_w = torch::tensor({2.0, -3.4}, options);
std::cout << true_w.size(0) << std::endl;
float true_b = 4.2;
int64_t num_samples = 20;

std::pair<torch::Tensor, torch::Tensor> data_and_label = synthetic_data(true_w, true_b, num_samples);

2


In [5]:
torch::Tensor X = data_and_label.first;
std::cout << X << std::endl;

torch::Tensor y = data_and_label.second;
std::cout << X << std::endl;

-0.9821 -2.5545
 1.2846 -0.1736
-1.4975  0.3841
 0.1357 -1.9922
-1.0394 -0.6618
-1.0471  1.6299
-0.3299 -0.7295
 1.5741  0.6600
 0.8150  0.5050
 0.9202  0.7417
-0.1508 -0.0004
-0.9096 -1.0994
-1.5390  0.6146
-1.0026  0.2359
-1.1073  0.2167
-0.3060  0.9002
 0.3151 -0.9156
-1.1112 -0.3220
 0.4563  1.4228
-0.5810  0.2224
[ CPUFloatType{20,2} ]
-0.9821 -2.5545
 1.2846 -0.1736
-1.4975  0.3841
 0.1357 -1.9922
-1.0394 -0.6618
-1.0471  1.6299
-0.3299 -0.7295
 1.5741  0.6600
 0.8150  0.5050
 0.9202  0.7417
-0.1508 -0.0004
-0.9096 -1.0994
-1.5390  0.6146
-1.0026  0.2359
-1.1073  0.2167
-0.3060  0.9002
 0.3151 -0.9156
-1.1112 -0.3220
 0.4563  1.4228
-0.5810  0.2224
[ CPUFloatType{20,2} ]


In [6]:
std::cout << torch::ones({X.size(0),1});

 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[ CPUFloatType{20,1} ]

In [7]:
torch::Tensor X_with_one = torch::cat({torch::ones({X.size(0),1}), X},1);

In [8]:
std::cout << X_with_one << std::endl;

 1.0000 -0.9821 -2.5545
 1.0000  1.2846 -0.1736
 1.0000 -1.4975  0.3841
 1.0000  0.1357 -1.9922
 1.0000 -1.0394 -0.6618
 1.0000 -1.0471  1.6299
 1.0000 -0.3299 -0.7295
 1.0000  1.5741  0.6600
 1.0000  0.8150  0.5050
 1.0000  0.9202  0.7417
 1.0000 -0.1508 -0.0004
 1.0000 -0.9096 -1.0994
 1.0000 -1.5390  0.6146
 1.0000 -1.0026  0.2359
 1.0000 -1.1073  0.2167
 1.0000 -0.3060  0.9002
 1.0000  0.3151 -0.9156
 1.0000 -1.1112 -0.3220
 1.0000  0.4563  1.4228
 1.0000 -0.5810  0.2224
[ CPUFloatType{20,3} ]


In [9]:
//torch::Tensor a = torch::randn({3,3});
//std::cout << torch::linalg::det(a);

## step2 use torch::linalg::lstsq to get regression coefficients

In [10]:
std::tuple<torch::Tensor,torch::Tensor,torch::Tensor,torch::Tensor> output = torch::linalg::lstsq(X_with_one,y, c10::nullopt, c10::nullopt);

In [11]:
torch::Tensor intercepts_and_coefficients = std::get<0>(output);
std::cout << intercepts_and_coefficients << std::endl;

 4.1999
 2.0007
-3.4013
[ CPUFloatType{3,1} ]


In [12]:
torch::Tensor y_pred_ts = X_with_one.matmul(intercepts_and_coefficients);

In [13]:
std::cout << y_pred_ts << std::endl;

 10.9235
  7.3608
 -0.1028
 11.2476
  4.3716
 -3.4388
  6.0210
  5.1044
  4.1128
  3.5183
  3.8997
  6.1194
 -0.9699
  1.3916
  1.2473
  0.5260
  7.9445
  3.0718
  0.2733
  2.2811
[ CPUFloatType{20,1} ]


In [14]:
torch::Tensor residuals = y - y_pred_ts;
std::cout << residuals << std::endl;

0.01 *
 0.9013
 -0.8118
 -0.4370
  0.6343
  0.5330
  1.0675
  0.0598
 -0.5617
  1.7832
 -0.2761
 -1.4416
 -1.2063
 -0.0428
  1.7338
 -2.2344
  1.2711
  0.2386
 -0.3345
 -0.1154
 -0.7614
[ CPUFloatType{20,1} ]


## appendix: understand c10::optional

In [16]:
c10::optional<double> a = c10::nullopt;
std::cout << a.has_value() << std::endl;

c10::optional<double> b(-1.1);
std::cout << b.has_value() << std::endl;;
std::cout << b.value() << std::endl;

0
1
-1.1
